# Common Crawl → Financial Features Pipeline

## Context
This notebook directly addresses the Numerai job description, which mentions "processing Common Crawl data" as a core responsibility. Common Crawl is the largest publicly available web crawl dataset — petabytes of HTML from billions of web pages.

The challenge: **extract financially relevant text from this haystack, map it to stock tickers, and produce clean features — all while maintaining strict point-in-time correctness.**

## My Infrastructure Experience
- **NDIF/NNsight** (ICLR 2025): Built Ray GCS Service backend with AWS object storage and VLLM for inference. Scaled to handle thousands of concurrent model probing requests.
- **NeuroData Lab**: Optimized a diffusion MRI pipeline with Kubernetes, Docker, and AWS Batch. Halved runtime, cut cloud costs 40%.
- **Creyon Bio**: Scaled data processing pipelines for protein sequence analysis.

## Pipeline Architecture
```
Common Crawl (WARC files, ~3TB/month)
    → Filter: financial content classifier
    → Parse: extract clean text from HTML
    → Entity Link: map text → stock tickers
    → Feature Extract: sentiment, embeddings, linguistic features (NB01-07)
    → Temporal Aggregate: daily text → weekly features (match Numerai eras)
    → Output: per-ticker feature vectors for Numerai Signals
```

In [ ]:
import re
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from collections import defaultdict, Counter
from urllib.parse import urlparse

## 1. Understanding Common Crawl Data

Common Crawl stores web pages in WARC (Web ARChive) format. Each monthly crawl contains:
- ~3 billion web pages
- ~200-400 TB compressed
- Available for free on S3: `s3://commoncrawl/`

Each WARC record contains:
- URL of the crawled page
- HTTP response headers
- HTML content
- Crawl timestamp (critical for point-in-time correctness)

In [ ]:
# Simulated Common Crawl WARC records
# In production: use warcio library to read actual WARC files from S3
# Example: aws s3 ls s3://commoncrawl/crawl-data/CC-MAIN-2024-10/segments/
simulated_warc_records = [
    {
        "url": "https://www.reuters.com/technology/apple-reports-record-q1-revenue-2024-01-25/",
        "crawl_timestamp": "2024-01-26T03:14:22Z",
        "content_type": "text/html",
        "html": "<html><head><title>Apple Reports Record Q1 Revenue</title></head><body><article><h1>Apple Reports Record Q1 Revenue Driven by iPhone 15</h1><p>Apple Inc reported record first-quarter revenue of $119.6 billion on Thursday, driven by strong iPhone 15 sales and growing services revenue. The tech giant beat analyst expectations of $117.9 billion.</p><p>CEO Tim Cook said the company saw particular strength in emerging markets, with India revenue hitting an all-time record.</p></article></body></html>",
    },
    {
        "url": "https://www.bloomberg.com/news/nvidia-datacenter-revenue-surge",
        "crawl_timestamp": "2024-01-26T05:22:11Z",
        "content_type": "text/html",
        "html": "<html><body><article><h1>NVIDIA Revenue Surges 400% as AI Demand Explodes</h1><p>NVIDIA Corp reported that data center revenue increased 409% year-over-year to $18.4 billion, far exceeding analyst estimates. The chipmaker's GPUs have become essential infrastructure for training large language models.</p><p>CEO Jensen Huang announced the next-generation Blackwell GPU architecture.</p></article></body></html>",
    },
    {
        "url": "https://finance.yahoo.com/news/tesla-recalls-vehicles",
        "crawl_timestamp": "2024-01-26T08:45:33Z",
        "content_type": "text/html",
        "html": "<html><body><div class='article'><h1>Tesla Recalls 2 Million Vehicles Over Autopilot Safety</h1><p>Tesla Inc is recalling approximately 2 million vehicles in the United States to install new safeguards in its Autopilot driver-assistance system, according to NHTSA.</p></div></body></html>",
    },
    {
        "url": "https://www.recipe-blog.com/best-chocolate-cake/",
        "crawl_timestamp": "2024-01-26T02:11:05Z",
        "content_type": "text/html",
        "html": "<html><body><h1>Best Chocolate Cake Recipe</h1><p>This rich chocolate cake uses premium cocoa powder and buttermilk for an incredibly moist texture.</p></body></html>",
    },
    {
        "url": "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=AAPL",
        "crawl_timestamp": "2024-01-26T12:00:00Z",
        "content_type": "text/html",
        "html": "<html><body><h1>EDGAR Company Search</h1><p>Apple Inc (AAPL) - Latest filings available.</p></body></html>",
    },
    {
        "url": "https://www.cnbc.com/jpmorgan-earnings-q4",
        "crawl_timestamp": "2024-01-26T06:33:22Z",
        "content_type": "text/html",
        "html": "<html><body><article><h1>JPMorgan Reports Strong Q4 Earnings</h1><p>JPMorgan Chase & Co reported fourth-quarter profit that beat expectations as the largest U.S. bank benefited from higher interest rates and strong trading revenue. Net income rose to $9.3 billion from $3.57 billion.</p></article></body></html>",
    },
    {
        "url": "https://www.espn.com/nfl/story/playoff-results",
        "crawl_timestamp": "2024-01-26T01:00:00Z",
        "content_type": "text/html",
        "html": "<html><body><h1>NFL Playoff Results</h1><p>The Kansas City Chiefs advanced to the conference championship with a thrilling overtime victory.</p></body></html>",
    },
    {
        "url": "https://www.ft.com/content/microsoft-openai-investment",
        "crawl_timestamp": "2024-01-26T09:15:44Z",
        "content_type": "text/html",
        "html": "<html><body><article><h1>Microsoft Deepens OpenAI Partnership with $10B Investment</h1><p>Microsoft Corp has agreed to invest an additional $10 billion in OpenAI, strengthening its position in the artificial intelligence race against Google and Amazon.</p></article></body></html>",
    },
    {
        "url": "https://www.investopedia.com/terms/p/price-earnings-ratio.asp",
        "crawl_timestamp": "2024-01-26T04:20:00Z",
        "content_type": "text/html",
        "html": "<html><body><h1>Price-to-Earnings Ratio (P/E)</h1><p>The price-to-earnings ratio is a financial metric used to value a company. It is calculated by dividing the stock price by earnings per share.</p></body></html>",
    },
]

print(f"Simulated WARC records: {len(simulated_warc_records)}")
for r in simulated_warc_records:
    domain = urlparse(r["url"]).netloc
    print(f"  [{domain}] {r['url'][:70]}...")

## 2. Step 1: Filter for Financial Content
Most of Common Crawl is irrelevant (recipes, sports, etc). We need a fast filter.

In [ ]:
# Financial content filtering strategy:
# Level 1: URL-based filtering (fast, no parsing needed)
# Level 2: Keyword-based filtering (requires text extraction)
# Level 3: ML classifier (most accurate, most expensive)

FINANCIAL_DOMAINS = {
    "reuters.com", "bloomberg.com", "cnbc.com", "ft.com", "wsj.com",
    "finance.yahoo.com", "marketwatch.com", "seekingalpha.com", "sec.gov",
    "businessinsider.com", "barrons.com", "economist.com", "investopedia.com",
    "thestreet.com", "morningstar.com", "fool.com", "benzinga.com",
}

FINANCIAL_URL_PATTERNS = [
    r"finance", r"business", r"market", r"stock", r"invest",
    r"earnings", r"revenue", r"economy", r"trading",
]

FINANCIAL_KEYWORDS = {
    "revenue", "earnings", "profit", "loss", "stock", "shares", "dividend",
    "quarterly", "fiscal", "billion", "million", "analyst", "forecast",
    "market", "trading", "investor", "shareholder", "CEO", "IPO",
    "acquisition", "merger", "SEC", "filing", "guidance", "margin",
    "growth", "decline", "beat", "miss", "estimate", "consensus",
}

def is_financial_url(url):
    """Level 1: Fast URL-based filter."""
    parsed = urlparse(url)
    domain = parsed.netloc.replace("www.", "")
    
    # Check known financial domains
    if domain in FINANCIAL_DOMAINS:
        return True, "domain_match"
    
    # Check URL patterns
    url_lower = url.lower()
    for pattern in FINANCIAL_URL_PATTERNS:
        if re.search(pattern, url_lower):
            return True, "url_pattern"
    
    return False, "no_match"

def is_financial_content(text, threshold=3):
    """Level 2: Keyword-based content filter."""
    words = set(re.findall(r'\b\w+\b', text.lower()))
    matches = words & FINANCIAL_KEYWORDS
    return len(matches) >= threshold, matches

def filter_pipeline(record):
    """Multi-level financial content filter."""
    # Level 1: URL check (cheapest)
    url_match, url_reason = is_financial_url(record["url"])
    if not url_match:
        # Level 2: Content check (need to parse HTML first)
        text = re.sub(r'<[^>]+>', ' ', record["html"])  # strip HTML
        content_match, keywords = is_financial_content(text)
        if not content_match:
            return False, "filtered_non_financial"
        return True, f"content_keywords: {keywords}"
    return True, f"url: {url_reason}"

# Apply filter
print("Filtering WARC records:")
print("-" * 80)
for record in simulated_warc_records:
    passed, reason = filter_pipeline(record)
    domain = urlparse(record["url"]).netloc
    status = "PASS" if passed else "SKIP"
    print(f"  [{status}] {domain:<35} | {reason}")

passed_records = [r for r in simulated_warc_records if filter_pipeline(r)[0]]
print(f"\nPassed: {len(passed_records)}/{len(simulated_warc_records)} records")

## 3. Step 2: Parse HTML → Clean Text

In [ ]:
def extract_text(html):
    """Extract clean text from HTML. In production, use BeautifulSoup or trafilatura."""
    # Remove script and style tags
    text = re.sub(r'<script[^>]*>.*?</script>', '', html, flags=re.DOTALL)
    text = re.sub(r'<style[^>]*>.*?</style>', '', html, flags=re.DOTALL)
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', ' ', text)
    # Clean whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def extract_title(html):
    """Extract title from HTML."""
    match = re.search(r'<title[^>]*>(.*?)</title>', html, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(1).strip()
    match = re.search(r'<h1[^>]*>(.*?)</h1>', html, re.DOTALL | re.IGNORECASE)
    if match:
        return re.sub(r'<[^>]+>', '', match.group(1)).strip()
    return ""

# Parse passed records
parsed_records = []
for record in passed_records:
    parsed = {
        "url": record["url"],
        "crawl_timestamp": record["crawl_timestamp"],
        "title": extract_title(record["html"]),
        "text": extract_text(record["html"]),
        "domain": urlparse(record["url"]).netloc.replace("www.", ""),
    }
    parsed_records.append(parsed)

parsed_df = pd.DataFrame(parsed_records)
print("Parsed financial content:")
for _, row in parsed_df.iterrows():
    print(f"\n  [{row['domain']}] {row['title']}")
    print(f"  Text: {row['text'][:120]}...")
    print(f"  Crawled: {row['crawl_timestamp']}")

## 4. Step 3: Entity Linking (Text → Stock Tickers)
Map company mentions to stock tickers. This is a critical and non-trivial step.

In [ ]:
# Entity linking: company name → stock ticker
# In production: use a comprehensive database + fuzzy matching + NER
ENTITY_DB = {
    # Company name variants → canonical ticker
    "apple": "AAPL", "apple inc": "AAPL",
    "nvidia": "NVDA", "nvidia corp": "NVDA",
    "tesla": "TSLA", "tesla inc": "TSLA",
    "microsoft": "MSFT", "microsoft corp": "MSFT",
    "jpmorgan": "JPM", "jpmorgan chase": "JPM", "jp morgan": "JPM",
    "google": "GOOGL", "alphabet": "GOOGL",
    "amazon": "AMZN", "amazon.com": "AMZN",
    "meta": "META", "facebook": "META",
    "openai": None,  # Private company — not in Numerai's universe
}

def link_entities(text):
    """Map company mentions in text to stock tickers."""
    text_lower = text.lower()
    found_tickers = set()
    found_companies = []
    
    # Sort by length (longest first) to avoid partial matches
    for company, ticker in sorted(ENTITY_DB.items(), key=lambda x: -len(x[0])):
        if company in text_lower and ticker is not None:
            found_tickers.add(ticker)
            found_companies.append((company, ticker))
    
    return list(found_tickers), found_companies

# Link entities in parsed records
for i, row in parsed_df.iterrows():
    tickers, companies = link_entities(row["text"])
    parsed_df.at[i, "tickers"] = tickers
    parsed_df.at[i, "n_tickers"] = len(tickers)

print("Entity Linking Results:")
for _, row in parsed_df.iterrows():
    tickers = row["tickers"] if isinstance(row["tickers"], list) else []
    print(f"  [{', '.join(tickers) if tickers else 'NONE'}] {row['title'][:60]}")

## 5. Step 4: Point-in-Time Correctness (CRITICAL)
This is the most important engineering constraint. Features must only use information available at prediction time.

In [ ]:
# Point-in-time correctness demonstration
# 
# KEY RULE: Use the CRAWL TIMESTAMP, not the publication date
# Why? The crawl timestamp is when the information was definitely available
# Publication dates can be:
#   - Missing or unreliable
#   - Backdated
#   - Different from when the content was actually accessible
#
# Numerai eras are weekly (Friday close). Features for era T must only use
# data crawled BEFORE era T's prediction deadline.

def assign_to_era(crawl_timestamp, era_schedule):
    """Assign a document to the latest era it's available for."""
    crawl_dt = datetime.fromisoformat(crawl_timestamp.replace("Z", "+00:00")).replace(tzinfo=None)
    for era_date in sorted(era_schedule, reverse=True):
        if crawl_dt <= era_date:
            continue
        return era_date
    return None

# Simulate weekly eras (Fridays)
era_dates = [datetime(2024, 1, 19), datetime(2024, 1, 26), datetime(2024, 2, 2)]

print("Point-in-Time Assignment:")
print("=" * 70)
for _, row in parsed_df.iterrows():
    era = assign_to_era(row["crawl_timestamp"], era_dates)
    era_str = era.strftime("%Y-%m-%d") if era else "BEFORE_FIRST_ERA"
    print(f"  Crawled: {row['crawl_timestamp'][:19]} \u2192 Era: {era_str}")
    print(f"    {row['title'][:60]}")
print()
print("RULE: Document crawled on Jan 26 is available for era Jan 26, NOT Jan 19")
print("This prevents look-ahead bias \u2014 the most common mistake in financial ML")

## 6. Step 5: Temporal Aggregation
Aggregate text features per ticker per era, with exponential decay weighting.

In [ ]:
# Temporal aggregation with exponential decay
# Recent documents are weighted more heavily than older ones

def exponential_decay_weight(days_old, half_life=7):
    """Weight that halves every `half_life` days."""
    return 2 ** (-days_old / half_life)

# Demonstrate decay function
days = np.arange(0, 60)
weights_7d = [exponential_decay_weight(d, half_life=7) for d in days]
weights_14d = [exponential_decay_weight(d, half_life=14) for d in days]
weights_30d = [exponential_decay_weight(d, half_life=30) for d in days]

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(days, weights_7d, label="Half-life: 7 days", linewidth=2)
ax.plot(days, weights_14d, label="Half-life: 14 days", linewidth=2)
ax.plot(days, weights_30d, label="Half-life: 30 days", linewidth=2)
ax.set_xlabel("Days Since Document Crawled")
ax.set_ylabel("Weight")
ax.set_title("Exponential Decay Weighting for Temporal Aggregation")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Design choice: half-life should match the signal horizon")
print("  - News sentiment: 7-day half-life (decays fast)")
print("  - SEC filing features: 30-90 day half-life (changes slowly)")
print("  - Numerai targets are 20-60 day returns \u2192 14-30 day half-life is reasonable")

## 7. Scaling Architecture

In [ ]:
# Scaling calculations for Common Crawl processing

# Monthly Common Crawl stats (approximate)
monthly_warc_records = 3_000_000_000  # 3 billion pages
avg_html_size_kb = 50
monthly_raw_size_tb = monthly_warc_records * avg_html_size_kb / 1e9  # ~150 TB

# After financial filtering (estimated 0.5-2% of web is financial)
financial_pct = 0.01  # 1% conservative estimate
monthly_financial_records = monthly_warc_records * financial_pct
monthly_financial_size_tb = monthly_raw_size_tb * financial_pct

# Numerai stock universe
numerai_stocks = 5000
us_stocks = 3000  # rough US portion

# Processing costs (rough estimates)
filter_cost_per_1m = 0.50  # URL filtering is cheap
parse_cost_per_1m = 2.00   # HTML parsing
ner_cost_per_1m = 10.00    # Named entity recognition
embed_cost_per_1m = 50.00  # Sentence embedding inference (GPU)

total_records_millions = monthly_financial_records / 1e6
total_cost = (filter_cost_per_1m + parse_cost_per_1m + ner_cost_per_1m + embed_cost_per_1m) * total_records_millions

print("=" * 60)
print("COMMON CRAWL SCALING ANALYSIS")
print("=" * 60)
print(f"\nMonthly Common Crawl:")
print(f"  Total pages:      {monthly_warc_records/1e9:.1f} billion")
print(f"  Raw size:          ~{monthly_raw_size_tb:.0f} TB")
print(f"\nAfter Financial Filtering ({financial_pct:.1%}):")
print(f"  Financial pages:   {monthly_financial_records/1e6:.0f} million")
print(f"  Financial size:    ~{monthly_financial_size_tb:.1f} TB")
print(f"\nEstimated Monthly Processing Cost:")
print(f"  URL filtering:     ${filter_cost_per_1m * total_records_millions:,.0f}")
print(f"  HTML parsing:      ${parse_cost_per_1m * total_records_millions:,.0f}")
print(f"  NER/Entity link:   ${ner_cost_per_1m * total_records_millions:,.0f}")
print(f"  Embedding (GPU):   ${embed_cost_per_1m * total_records_millions:,.0f}")
print(f"  \u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500")
print(f"  Total:             ${total_cost:,.0f}/month")
print(f"\nTarget: {numerai_stocks:,} stocks with weekly features")
print(f"  \u2192 {monthly_financial_records/numerai_stocks:.0f} articles per stock per month")

# Architecture diagram (text-based)
print("\n" + "=" * 60)
print("ARCHITECTURE")
print("=" * 60)
print("""
\u250c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2510
\u2502                   Common Crawl (S3)                      \u2502
\u2502              s3://commoncrawl/crawl-data/                \u2502
\u2514\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u252c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2518
                       \u2502 WARC files (~3B pages/month)
                       \u25bc
\u250c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2510
\u2502          Stage 1: URL Filter (Ray / Spark)                \u2502
\u2502  \u2022 Domain whitelist (reuters, bloomberg, etc.)            \u2502
\u2502  \u2022 URL pattern matching                                   \u2502
\u2502  \u2022 Drops ~98% of records                                  \u2502
\u2502  \u2022 Cost: ~$0.50/M records                                \u2502
\u2514\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u252c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2518
                       \u2502 ~30M financial pages
                       \u25bc
\u250c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2510
\u2502          Stage 2: HTML Parse + Content Filter             \u2502
\u2502  \u2022 trafilatura / BeautifulSoup                           \u2502
\u2502  \u2022 Keyword-based content classifier                       \u2502
\u2502  \u2022 Extract: title, text, metadata, crawl timestamp       \u2502
\u2502  \u2022 Cost: ~$2/M records                                   \u2502
\u2514\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u252c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2518
                       \u2502 ~15M clean text documents
                       \u25bc
\u250c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2510
\u2502          Stage 3: Entity Linking (GPU)                    \u2502
\u2502  \u2022 spaCy NER / fine-tuned NER model                     \u2502
\u2502  \u2022 Company name \u2192 Ticker mapping                         \u2502
\u2502  \u2022 Fuzzy matching for name variants                      \u2502
\u2502  \u2022 Cost: ~$10/M records                                  \u2502
\u2514\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u252c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2518
                       \u2502 ~10M ticker-linked documents
                       \u25bc
\u250c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2510
\u2502          Stage 4: Feature Extraction (GPU)                \u2502
\u2502  \u2022 FinBERT sentiment (NB01)                              \u2502
\u2502  \u2022 Sentence embeddings (NB02)                            \u2502
\u2502  \u2022 LM linguistic features (NB03)                         \u2502
\u2502  \u2022 Fine-tuned LLM embeddings (NB07)                      \u2502
\u2502  \u2022 Cost: ~$50/M records (GPU inference)                  \u2502
\u2514\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u252c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2518
                       \u2502 Per-document feature vectors
                       \u25bc
\u250c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2510
\u2502          Stage 5: Temporal Aggregation                    \u2502
\u2502  \u2022 Group by (ticker, era_week)                           \u2502
\u2502  \u2022 Exponential decay weighting                           \u2502
\u2502  \u2022 Statistics: mean, std, min, max, count                \u2502
\u2502  \u2022 Point-in-time enforcement                             \u2502
\u2514\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u252c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2518
                       \u2502 Per-ticker, per-era feature vectors
                       \u25bc
\u250c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2510
\u2502          Numerai Signals Submission                       \u2502
\u2502  \u2022 5,000 tickers \u00d7 N features                           \u2502
\u2502  \u2022 Rank-normalized to [0, 1]                             \u2502
\u2502  \u2022 Weekly submission                                      \u2502
\u2514\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2518
""")

## Discussion & Interview Talking Points

### Why I Can Do This
- **NDIF** (ICLR 2025): I built a distributed inference platform using Ray, AWS, and VLLM that handles thousands of concurrent probing requests. Common Crawl processing is a similar distributed data pipeline.
- **NeuroData MRI Pipeline**: Optimized a Kubernetes + AWS Batch pipeline for processing neuroimaging data. Halved runtime, cut cloud costs 40%. Same principles apply: parallelism, fault tolerance, cost optimization.
- **Scale mindset**: I've worked with petabyte-scale data (neuroimaging), distributed training, and production ML systems.

### Key Engineering Challenges
1. **Deduplication**: Common Crawl has many duplicate pages across monthly crawls. Need MinHash/SimHash dedup.
2. **Language filtering**: Most of CC is English, but need to handle multilingual content for global stocks.
3. **Rate limiting**: If supplementing CC with live API calls, respect rate limits.
4. **Storage**: Processed features for 5,000 stocks x 52 weeks x N features = manageable, but raw text is huge.
5. **Monitoring**: Need observability for a pipeline this complex (logging, alerting, data quality checks).

### Technology Stack I'd Use
| Component | Technology | Why |
|-----------|-----------|-----|
| Orchestration | Ray / Spark | Distributed processing, fault tolerance |
| Storage | S3 + Parquet | Columnar format, cheap storage |
| Compute | AWS Batch / EC2 | Spot instances for cost savings |
| GPU inference | VLLM / TGI | Batched inference for embeddings |
| Monitoring | W&B / MLflow | Already experienced with both |
| Scheduling | Airflow / Prefect | Weekly pipeline runs |

### Extensions (TODO)
- [ ] Download and process a real CC WARC segment (warcio + boto3)
- [ ] Implement MinHash deduplication for cross-crawl dedup
- [ ] Build a financial content classifier (fine-tune BERT on financial vs non-financial)
- [ ] Add comprehensive entity linking with fuzzy matching (fuzzywuzzy + company database)
- [ ] Estimate actual AWS costs for monthly processing
- [ ] Set up an Airflow DAG for weekly pipeline execution